In [1]:
import sys
sys.path.append('/Eryn')

In [2]:
# import torch
import multiprocessing
 
from eryn.ensemble import EnsembleSampler
from eryn.state import State
from eryn.prior import ProbDistContainer, uniform_dist
from eryn.utils import TransformContainer
from eryn.moves import GaussianMove, StretchMove, CombineMove
from eryn.utils.utility import groups_from_inds

import matplotlib.pyplot as plt
import numpy as np

# set random seed
np.random.seed(42)

# cpu_count = multiprocessing.cpu_count()
# print("CPU数量:", cpu_count)
# # 查看系统中可用的 GPU 数量
# num_gpus = torch.cuda.device_count()
# print("GPU数量：", num_gpus)

# 该测试仅改动参数维度、nwalker和temper的数量

ndim = 40
nwalkers = 800
ntemps = 40

nsteps =5
burn = 1

thin_by = 1

In [3]:
# Gaussian likelihood
def log_like_fn(x, mu, invcov):
    diff = x - mu
    return -0.5 * (diff * np.dot(invcov, diff.T).T).sum()


In [4]:
# mean
means = np.zeros(ndim)  # np.random.rand(ndim)

# define covariance matrix
cov = np.diag(np.ones(ndim))
invcov = np.linalg.inv(cov)

In [5]:
# set prior limits
lims = 5.0
priors_in = {i: uniform_dist(-lims + means[i], lims + means[i]) for i in range(ndim)}
priors = ProbDistContainer(priors_in)


In [6]:
# fill kwargs dictionary
tempering_kwargs=dict(ntemps=ntemps)

# randomize throughout prior
coords = priors.rvs(size=(ntemps, nwalkers,))

# initialize sampler
ensemble_pt = EnsembleSampler(
    nwalkers,
    ndim,
    log_like_fn,
    priors,
    args=[means, cov],
    tempering_kwargs=tempering_kwargs,
    # pool=True,
    # vectorize=True,
)
ensemble_pt.run_mcmc(coords, nsteps, burn=burn, progress=True, thin_by=thin_by)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.94it/s]


In [7]:
initial_state = ensemble_pt._previous_state
coords = initial_state.branches_coords
inds = initial_state.branches_inds
# initial_state.log_like.shape, initial_state.log_prior.shape, coords['model_0'].shape

In [8]:
import time
# Groundtruth

start_time =time.time()
log_like = ensemble_pt.compute_log_like(
        coords,
        inds=inds,
        logp=initial_state.log_prior,
        supps=initial_state.supplimental,  # only used if self.provide_supplimental is True
        branch_supps=initial_state.branches_supplimental,  # only used if self.provide_supplimental is True
    )[0]
print(f"Groundtruth code time：{time.time()-start_time}")
log_like.shape
print(log_like)

Groundtruth code time：0.9164457321166992
[[ -35.97364673  -37.76164563  -42.8906489  ...  -36.12542081
   -30.83841615  -13.55685512]
 [ -44.96763615  -49.27304825  -54.64686885 ...  -49.80771132
   -53.3003641   -35.49659629]
 [ -82.41825586  -78.58970686  -54.74686663 ...  -58.31230148
   -69.50697803  -50.64450974]
 ...
 [-179.71717093 -127.07477839 -150.07498884 ... -191.96325102
  -195.52117062 -179.58485631]
 [-167.5484348  -133.59299462 -138.01766934 ... -141.13926265
  -175.16417022 -179.15357214]
 [-158.99261428 -180.97518019 -188.36775529 ... -201.69256795
  -142.3314211  -168.48709665]]


In [9]:

#当ndim = 100 nwalkers = 1000  ntemps = 100 时
# 首先报错提醒cpu的数量不能全部占用，减少cpu后报错提示空间溢出来了。
# File D:\Anaconda\envs\taiji\Lib\site-packages\ray\_private\utils.py:749, in check_oversized_function(pickled, name, obj_type, worker)
#     737 else:
#     738     error = (
#     739         "The {} {} is too large ({} MiB > FUNCTION_SIZE_ERROR_THRESHOLD={}"
#     740         " MiB). Check that its definition is not implicitly capturing a "
#    (...)
#     747         ray_constants.FUNCTION_SIZE_ERROR_THRESHOLD // (1024 * 1024),
#     748     )
# --> 749     raise ValueError(error)

# ValueError: The remote function eryn.ensemble.parallel_log_like_fn is too large (392 MiB > FUNCTION_SIZE_ERROR_THRESHOLD=95 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
